In [ ]:
import gymnasium as gym 
from gymnasium import Env
import numpy as np
import matplotlib.pyplot as plt
import random
import math

# # Set the global seed value
# global_seed = random.randint(0, 1000)
# np.random.seed(global_seed)
# random.seed(global_seed)

class missile_interception_3d(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.MultiDiscrete([5, 5])
        self.observation_space = None
        self.np_random = np.random.RandomState()
        self.time_step = 0.5  # seconds
        self.g = 9.81  # gravity
        self.navigation_constant = 4  # ProNav gain

        # Target box dimensions
        self.targetbox_x_min = -15000
        self.targetbox_x_max = 15000
        self.targetbox_y_min = -15000
        self.targetbox_y_max = 15000

    def generate_enemy_missile(self):
        range_min, range_max = 70000, 1000000
        self.attack_target_x = self.np_random.uniform(self.targetbox_x_min, self.targetbox_x_max)
        self.attack_target_y = self.np_random.uniform(self.targetbox_y_min, self.targetbox_y_max)

        self.enemy_launch_angle = self.np_random.uniform(0, 2 * np.pi)
        # print("(1) ENEMY LAUNCH ANGLE: ", self.enemy_launch_angle)
        # print("(1) ENEMY LAUNCH ANGLE DEGREES: ", np.degrees(self.enemy_launch_angle))
        self.enemy_theta = self.np_random.uniform(0.523599, 1.0472)
        lower_limit = np.sqrt((range_min * self.g) / np.sin(2 * self.enemy_theta))
        upper_limit = np.sqrt((range_max * self.g) / np.sin(2 * self.enemy_theta))
        self.enemy_initial_velocity = self.np_random.uniform(lower_limit, upper_limit)
        range = self.enemy_initial_velocity * np.cos(self.enemy_theta) * (2 * self.enemy_initial_velocity * np.sin(self.enemy_theta) / self.g)

        self.enemy_launch_x = self.attack_target_x + range * np.cos(self.enemy_launch_angle)
        self.enemy_launch_y = self.attack_target_y + range * np.sin(self.enemy_launch_angle)
        self.enemy_z = 0

        self.enemy_x = self.enemy_launch_x
        self.enemy_y = self.enemy_launch_y

        # print("(2) ENEMY LAUNCH ANGLE: ", self.enemy_launch_angle)
        # print("(2) ENEMY LAUNCH ANGLE DEGREES: ", np.degrees(self.enemy_launch_angle))

        self.enemy_azimuth = (self.enemy_launch_angle + np.pi) % (2 * np.pi)

        # print("(3) ENEMY AZIMUTH: ", self.enemy_azimuth)
        # print("(3) ENEMY AZIMUTH DEGREES: ", np.degrees(self.enemy_azimuth))

        # print("Enemy launch angle: ", self.enemy_launch_angle)
        # print("Enemy launch angle degrees: ", np.degrees(self.enemy_launch_angle))

        # print("ENEMY AZIMUTH: ", self.enemy_azimuth)
        # print("ENEMY AZIMUTH DEGREES: ", np.degrees(self.enemy_azimuth))

        # print("Enemy launch angle: ", self.enemy_launch_angle)
        # print("Enemy launch angle degrees: ", np.degrees(self.enemy_launch_angle))

    def generate_defense_missile(self):
        self.defense_launch_x = self.np_random.uniform(self.targetbox_x_min, self.targetbox_x_max)
        self.defense_launch_y = self.np_random.uniform(self.targetbox_y_min, self.targetbox_y_max)
        # print("self.enemy_launch_angle: ", self.enemy_launch_angle)
        self.defense_azimuth = self.np_random.uniform((self.enemy_launch_angle - 0.785398), (self.enemy_launch_angle + 0.785398))
        self.defense_theta = 0.785398
        self.defense_initial_velocity = 3000
        self.defense_x = self.defense_launch_x
        self.defense_y = self.defense_launch_y
        self.defense_z = 0

        # print("DEFENSE AZIMUTH: ", self.defense_azimuth)
        # print("DEFENSE AZIMUTH DEGREES: ", np.degrees(self.defense_azimuth))
        # print("Minus plus 45 degrees: ", np.degrees(self.enemy_azimuth - 0.785398), np.degrees(self.enemy_azimuth + 0.785398))

    def move_missiles(self):
        # Update defense missile velocity due to gravity (z-component)
        self.defense_velocity[2] -= self.g * self.time_step

        # Update defense missile position
        self.defense_x += self.defense_velocity[0] * self.time_step
        self.defense_y += self.defense_velocity[1] * self.time_step
        self.defense_z += self.defense_velocity[2] * self.time_step

        if self.defense_z < 0:
            print("DEFENSE MISSILE HIT GROUND")
            self.done = True

        # Update enemy missile velocity due to gravity
        self.enemy_velocity[2] -= self.g * self.time_step

        # Update enemy missile position
        self.enemy_x += self.enemy_velocity[0] * self.time_step
        self.enemy_y += self.enemy_velocity[1] * self.time_step
        self.enemy_z += self.enemy_velocity[2] * self.time_step

        if self.enemy_z < 0:
            print("ENEMY MISSILE HIT GROUND")
            self.done = True

    def calculate_pro_nav_acceleration(self):
        relative_position = np.array([
            self.enemy_x - self.defense_x,
            self.enemy_y - self.defense_y,
            self.enemy_z - self.defense_z
        ])
        # print("Relative position: ", relative_position)

        relative_velocity = self.enemy_velocity - self.defense_velocity
        range_to_target = np.linalg.norm(relative_position)

        # print("Range to target: ", range_to_target)
        if range_to_target == 0:
            self.done = True
            return np.array([0, 0, 0])  # No acceleration needed

        # Estimate time-to-go
        closing_velocity = -np.dot(relative_velocity, relative_position) / range_to_target
        Tgo = range_to_target / closing_velocity if closing_velocity != 0 else 1  # Avoid division by zero

        # Calculate Zero Effort Miss (ZEM)
        ZEM = relative_position + relative_velocity * Tgo

        # Calculate ProNav acceleration
        required_acceleration = (self.navigation_constant * ZEM) / (Tgo**2)
        return required_acceleration

    def step(self, action):
        acceleration_command = self.calculate_pro_nav_acceleration()
        # print("Acceleration command: ", acceleration_command)
        self.defense_velocity += acceleration_command * self.time_step
        # print("Updated defense velocity: ", self.defense_velocity)
        self.move_misses()
        self.enemy_path.append([self.enemy_x, self.enemy_y, self.enemy_z])
        self.defense_path.append([self.defense_x, self.defense_y, self.defense_z])

                # Calculate relative distance and store it
        relative_distance = np.linalg.norm([
            self.enemy_x - self.defense_x,
            self.enemy_y - self.defense_y,
            self.enemy_z - self.defense_z
        ])
        self.relative_distances.append(relative_distance)
        self.times.append(self.t)

        collision_distance = 10  # meters
        if relative_distance <= collision_distance:
            self.success = True
            self.done = True

        self.t += self.time_step

    def reset(self):
        self.done = False
        self.reward = 0
        self.enemy_path, self.defense_path = [], []
        self.relative_distances = []
        self.times = []
        self.generate_enemy_missile()
        self.generate_defense_missile()
        self.t = 0
        self.defense_velocity = np.array([
            self.defense_initial_velocity * np.cos(self.defense_azimuth) * np.cos(self.defense_theta),
            self.defense_initial_velocity * np.sin(self.defense_azimuth) * np.cos(self.defense_theta),
            self.defense_initial_velocity * np.sin(self.defense_theta)
        ])
        self.enemy_velocity = np.array([
            self.enemy_initial_velocity * np.cos(self.enemy_azimuth) * np.cos(self.enemy_theta),
            self.enemy_initial_velocity * np.sin(self.enemy_azimuth) * np.cos(self.enemy_theta),
            self.enemy_initial_velocity * np.sin(self.enemy_theta)
        ])
        self.t = 0
        self.time_steps = []  # To store time steps for analysis
        self.success = False

    def adjust_time_step(self, relative_distance):
        if relative_distance > 100000:
            self.time_step = 1.0
        elif relative_distance > 50000:
            self.time_step = 0.5
        elif relative_distance > 10000:
            self.time_step = 0.1
        elif relative_distance > 1000:
            self.time_step = 0.01
        elif relative_distance > 100:
            self.time_step = 0.005
        else:
            self.time_step = 0.001
        self.time_steps.append(self.time_step)

    def graph_initial(self):
        plt.figure()
        plt.plot(self.enemy_x, self.enemy_y, 'ro', label="Enemy missile")
        plt.plot(self.defense_x, self.defense_y, 'bo', label="Defense missile")
        plt.plot(self.attack_target_x, self.attack_target_y, 'go', label="Attack target")
        plt.xlabel('X coordinate (m)')
        plt.ylabel('Y coordinate (m)')
        plt.title('Missile interception')
        plt.legend()
        plt.axis('equal')  # Ensures 1:1 aspect ratio
        plt.show()

    def render(self):
        enemy_path_array = np.array(self.enemy_path)
        defense_path_array = np.array(self.defense_path)
        print("enemy_path_array: ", enemy_path_array)
        print("defense_path_array: ", defense_path_array)    
        xe, ye, ze = enemy_path_array[:, 0], enemy_path_array[:, 1], enemy_path_array[:, 2]

        xd, yd, zd = defense_path_array[:, 0], defense_path_array[:, 1], defense_path_array[:, 2]

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(xe, ye, ze, label="Enemy missile trajectory", color='blue')
        ax.plot(xd, yd, zd, label="Defense missile trajectory", color='red')
        ax.scatter(xe[0], ye[0], ze[0], color='blue', s=50, label="Enemy Start")
        ax.scatter(xd[0], yd[0], zd[0], color='red', s=50, label="Defense Start")
        ax.set_xlabel('X coordinate (m)')
        ax.set_ylabel('Y coordinate (m)')
        ax.set_zlabel('Z coordinate (m)')
        ax.set_title('Missile Trajectory')
        ax.legend()
        plt.show()

# env = missile_interception_3d()
# env.reset()
# env.graph_initial()
# # env.render()

# # # # Simulation loop
# while not env.done:
#     print("-------------------------")
#     print("t: ", env.t)
#     print("enemy_x: ", env.enemy_x)
#     print("enemy_y: ", env.enemy_y)
#     print("enemy_z: ", env.enemy_z)
#     print("defense_x: ", env.defense_x)
#     print("defense_y: ", env.defense_y)
#     print("defense_z: ", env.defense_z)
#     print("-------------------------")
#     env.step([0, 0])  # Placeholder action, ProNav handles turning
#     env.t += env.time_step

# # # env.render()

num_simulations = 1000
final_relative_distances = []
success_count = 0

for _ in range(num_simulations):
    env = missile_interception_3d()
    env.reset()

    while not env.done:
        # Calculate relative distance
        relative_distance = np.linalg.norm([
            env.enemy_x - env.defense_x,
            env.enemy_y - env.defense_y,
            env.enemy_z - env.defense_z
        ])
        # Adjust time step based on relative distance
        env.adjust_time_step(relative_distance)
        env.step([0, 0])  # Placeholder action, ProNav handles turning
        env.t += env.time_step

    # print the minimum relative distance
    print("#########################################################")
    print("#########################################################")
    print("#########################################################")
    print("#########################################################")
    print("Missile Hit")
    print("Minimum relative distance: ", min(env.relative_distances))
    print("#########################################################")
    print("#########################################################")
    print("######################## #################################")
    print("#########################################################\n")


    # plt.figure()
    # plt.plot(env.times, env.relative_distances, label='Relative Distance')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Distance (m)')
    # plt.title('Relative Distance Between Missiles Over Time')
    # plt.legend()
    # plt.grid(True)
    # plt.show()

    final_relative_distances.append(env.relative_distances[-1])
    if env.success:
        success_count += 1

# Calculate average final relative distance
average_relative_distance = np.mean(final_relative_distances)
success_rate = (success_count / num_simulations) * 100

print(f"Average final relative distance over {num_simulations} simulations: {average_relative_distance:.2f} meters")
print(f"Interception success rate: {success_rate:.2f}%")

#########################################################
#########################################################
#########################################################
#########################################################
Minimum relative distance:  7.457940163235619
#########################################################
#########################################################
#########################################################
#########################################################

#########################################################
#########################################################
#########################################################
#########################################################
Minimum relative distance:  9.8999316096137
#########################################################
#########################################################
#########################################################
#######################################

In [24]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML, display

def update_paths(num, xe, ye, ze, xd, yd, zd, lines, ax):
    # Update the data in the trajectory plots
    lines[0].set_data_3d(xe[:num], ye[:num], ze[:num])
    lines[1].set_data_3d(xd[:num], yd[:num], zd[:num])

    # Rotate the camera angle
    ax.view_init(elev=10, azim=1 * num)  # Rotate slowly
    return lines

def animate_trajectories(enemy_path, defense_path, num_steps):
    enemy_path_array = np.array(enemy_path)
    defense_path_array = np.array(defense_path)

    xe, ye, ze = enemy_path_array[:, 0], enemy_path_array[:, 1], enemy_path_array[:, 2]
    xd, yd, zd = defense_path_array[:, 0], defense_path_array[:, 1], defense_path_array[:, 2]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim([np.min([xe, xd]), np.max([xe, xd])])
    ax.set_ylim([np.min([ye, yd]), np.max([ye, yd])])
    ax.set_zlim([np.min([ze, zd]), np.max([ze, zd])])

    # Plot initially without data
    line_enemy, = ax.plot([], [], [], 'b-', label="Enemy missile trajectory")
    line_defense, = ax.plot([], [], [], 'r-', label="Defense missile trajectory")
    lines = [line_enemy, line_defense]

    # Setting labels and titles
    ax.set_ylabel('Y coordinate (m)')
    ax.set_xlabel('X coordinate (m)')
    ax.set_zlabel('Z coordinate (m)')
    ax.set_title('Animated Missile Trajectory')
    ax.legend()

    # Creating the Animation object
    ani = animation.FuncAnimation(
        fig, update_paths, num_steps, fargs=(xe, ye, ze, xd, yd, zd, lines, ax),
        interval=20, blit=False)

    # Option 2: Display using HTML
    plt.close(fig)
    video = ani.to_html5_video()
    html = display(HTML(video))

    return html

num_steps = len(env.enemy_path)
animate_trajectories(env.enemy_path, env.defense_path, num_steps)

#dang
#dang 2


test


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.animation as animation
from IPython.display import HTML, display

def update_paths(num, xe, ye, ze, xd, yd, zd, lines, ax):
    # Update the data in the trajectory plots
    lines[0].set_data_3d(xe[:num], ye[:num], ze[:num])
    lines[1].set_data_3d(xd[:num], yd[:num], zd[:num])

    # Rotate the camera angle
    ax.view_init(elev=90, azim=0 * num)  # Rotate slowly
    return lines

def animate_trajectories(enemy_path, defense_path, num_steps):
    enemy_path_array = np.array(enemy_path)
    defense_path_array = np.array(defense_path)

    xe, ye, ze = enemy_path_array[:, 0], enemy_path_array[:, 1], enemy_path_array[:, 2]
    xd, yd, zd = defense_path_array[:, 0], defense_path_array[:, 1], defense_path_array[:, 2]

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_xlim([np.min([xe, xd]), np.max([xe, xd])])
    ax.set_ylim([np.min([ye, yd]), np.max([ye, yd])])
    ax.set_zlim([np.min([ze, zd]), np.max([ze, zd])])

    # Plot initially without data
    line_enemy, = ax.plot([], [], [], 'b-', label="Enemy missile trajectory")
    line_defense, = ax.plot([], [], [], 'r-', label="Defense missile trajectory")
    lines = [line_enemy, line_defense]

    # Setting labels and titles
    ax.set_xlabel('X coordinate (m)')
    ax.set_ylabel('Y coordinate (m)')
    ax.set_zlabel('Z coordinate (m)')
    ax.set_title('Animated Missile Trajectory')
    ax.legend()

    # Creating the Animation object
    ani = animation.FuncAnimation(
        fig, update_paths, num_steps, fargs=(xe, ye, ze, xd, yd, zd, lines, ax),
        interval=20, blit=False)

    # Option 2: Display using HTML
    plt.close(fig)
    video = ani.to_html5_video()
    html = display(HTML(video))

    return html

num_steps = len(env.enemy_path)
animate_trajectories(env.enemy_path, env.defense_path, num_steps)


: 

In [ ]:
import gymnasium as gym 
from gymnasium import Env
import numpy as np
import matplotlib.pyplot as plt
import random
import math

# # Set the global seed value
# global_seed = random.randint(0, 1000)
# np.random.seed(global_seed)
# random.seed(global_seed)

class missile_interception_3d(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.Box(low=-40.0, high=40.0, shape=(2, ), dtype=np.float32)
        self.observation_space = None
        self.np_random = np.random.RandomState()
        self.time_step = 0.5  # seconds
        self.g = 9.81  # gravity

        # Target box dimensions
        self.targetbox_x_min = -15000
        self.targetbox_x_max = 15000
        self.targetbox_y_min = -15000
        self.targetbox_y_max = 15000

    def generate_enemy_missile(self):
        range_min, range_max = 70000, 1000000
        self.attack_target_x = self.np_random.uniform(self.targetbox_x_min, self.targetbox_x_max)
        self.attack_target_y = self.np_random.uniform(self.targetbox_y_min, self.targetbox_y_max)

        self.enemy_launch_angle = self.np_random.uniform(0, 2 * np.pi)
        self.enemy_theta = self.np_random.uniform(0.523599, 1.0472)
        lower_limit = np.sqrt((range_min * self.g) / np.sin(2 * self.enemy_theta))
        upper_limit = np.sqrt((range_max * self.g) / np.sin(2 * self.enemy_theta))
        self.enemy_initial_velocity = self.np_random.uniform(lower_limit, upper_limit)
        range = self.enemy_initial_velocity * np.cos(self.enemy_theta) * (2 * self.enemy_initial_velocity * np.sin(self.enemy_theta) / self.g)

        self.enemy_launch_x = self.attack_target_x + range * np.cos(self.enemy_launch_angle)
        self.enemy_launch_y = self.attack_target_y + range * np.sin(self.enemy_launch_angle)
        self.enemy_z = 0

        self.enemy_x = self.enemy_launch_x
        self.enemy_y = self.enemy_launch_y

        self.enemy_azimuth = (self.enemy_launch_angle + np.pi) % (2 * np.pi)

    def generate_defense_missile(self):
        self.defense_launch_x = self.np_random.uniform(self.targetbox_x_min, self.targetbox_x_max)
        self.defense_launch_y = self.np_random.uniform(self.targetbox_y_min, self.targetbox_y_max)
        # print("self.enemy_launch_angle: ", self.enemy_launch_angle)
        self.defense_azimuth = self.np_random.uniform((self.enemy_launch_angle - 0.785398), (self.enemy_launch_angle + 0.785398))
        self.defense_theta = 0.785398
        self.defense_initial_velocity = 3000
        self.defense_x = self.defense_launch_x
        self.defense_y = self.defense_launch_y
        self.defense_z = 0

        self.defense_ax = 0
        self.defense_ay = 0
        self.defense_az = 0 
        
    def move_missiles(self, action):
        # Update defense missile velocity due to gravity (z-component)
        self.defense_bodyacc = [0, action[0] * self.g, action[1] * self.g]
        


        self.defense_velocity[2] -= self.g * self.time_step

        # Update defense missile position
        self.defense_x += self.defense_velocity[0] * self.time_step
        self.defense_y += self.defense_velocity[1] * self.time_step
        self.defense_z += self.defense_velocity[2] * self.time_step

        if self.defense_z < 0:
            print("DEFENSE MISSILE HIT GROUND")
            self.done = True

        # Update enemy missile velocity due to gravity
        self.enemy_velocity[2] -= self.g * self.time_step

        # Update enemy missile position
        self.enemy_x += self.enemy_velocity[0] * self.time_step
        self.enemy_y += self.enemy_velocity[1] * self.time_step
        self.enemy_z += self.enemy_velocity[2] * self.time_step

        if self.enemy_z < 0:
            print("ENEMY MISSILE HIT GROUND")
            self.done = True

    def calculate_pro_nav_acceleration(self):
        relative_position = np.array([
            self.enemy_x - self.defense_x,
            self.enemy_y - self.defense_y,
            self.enemy_z - self.defense_z
        ])
        # print("Relative position: ", relative_position)

        relative_velocity = self.enemy_velocity - self.defense_velocity
        range_to_target = np.linalg.norm(relative_position)

        # print("Range to target: ", range_to_target)
        if range_to_target == 0:
            self.done = True
            return np.array([0, 0, 0])  # No acceleration needed

        # Estimate time-to-go
        closing_velocity = -np.dot(relative_velocity, relative_position) / range_to_target
        Tgo = range_to_target / closing_velocity if closing_velocity != 0 else 1  # Avoid division by zero

        # Calculate Zero Effort Miss (ZEM)
        ZEM = relative_position + relative_velocity * Tgo

        # Calculate ProNav acceleration
        required_acceleration = (self.navigation_constant * ZEM) / (Tgo**2)
        return required_acceleration

    def step(self, action):
        acceleration_command = self.calculate_pro_nav_acceleration()
        # print("Acceleration command: ", acceleration_command)
        self.defense_velocity += acceleration_command * self.time_step
        # print("Updated defense velocity: ", self.defense_velocity)
        self.move_misses()
        self.enemy_path.append([self.enemy_x, self.enemy_y, self.enemy_z])
        self.defense_path.append([self.defense_x, self.defense_y, self.defense_z])

                # Calculate relative distance and store it
        relative_distance = np.linalg.norm([
            self.enemy_x - self.defense_x,
            self.enemy_y - self.defense_y,
            self.enemy_z - self.defense_z
        ])
        self.relative_distances.append(relative_distance)
        self.times.append(self.t)

        collision_distance = 10  # meters
        if relative_distance <= collision_distance:
            self.success = True
            self.done = True

        self.t += self.time_step

    def reset(self):
        self.done = False
        self.reward = 0
        self.enemy_path, self.defense_path = [], []
        self.relative_distances = []
        self.times = []
        self.generate_enemy_missile()
        self.generate_defense_missile()
        self.t = 0
        self.defense_velocity = np.array([
            self.defense_initial_velocity * np.cos(self.defense_azimuth) * np.cos(self.defense_theta),
            self.defense_initial_velocity * np.sin(self.defense_azimuth) * np.cos(self.defense_theta),
            self.defense_initial_velocity * np.sin(self.defense_theta)
        ])
        self.enemy_velocity = np.array([
            self.enemy_initial_velocity * np.cos(self.enemy_azimuth) * np.cos(self.enemy_theta),
            self.enemy_initial_velocity * np.sin(self.enemy_azimuth) * np.cos(self.enemy_theta),
            self.enemy_initial_velocity * np.sin(self.enemy_theta)
        ])
        self.t = 0
        self.time_steps = []  # To store time steps for analysis
        self.success = False

    def adjust_time_step(self, relative_distance):
        if relative_distance > 100000:
            self.time_step = 1.0
        elif relative_distance > 50000:
            self.time_step = 0.5
        elif relative_distance > 10000:
            self.time_step = 0.1
        elif relative_distance > 1000:
            self.time_step = 0.01
        elif relative_distance > 100:
            self.time_step = 0.005
        else:
            self.time_step = 0.001
        self.time_steps.append(self.time_step)

    def graph_initial(self):
        plt.figure()
        plt.plot(self.enemy_x, self.enemy_y, 'ro', label="Enemy missile")
        plt.plot(self.defense_x, self.defense_y, 'bo', label="Defense missile")
        plt.plot(self.attack_target_x, self.attack_target_y, 'go', label="Attack target")
        plt.xlabel('X coordinate (m)')
        plt.ylabel('Y coordinate (m)')
        plt.title('Missile interception')
        plt.legend()
        plt.axis('equal')  # Ensures 1:1 aspect ratio
        plt.show()

    def render(self):
        enemy_path_array = np.array(self.enemy_path)
        defense_path_array = np.array(self.defense_path)
        print("enemy_path_array: ", enemy_path_array)
        print("defense_path_array: ", defense_path_array)    
        xe, ye, ze = enemy_path_array[:, 0], enemy_path_array[:, 1], enemy_path_array[:, 2]

        xd, yd, zd = defense_path_array[:, 0], defense_path_array[:, 1], defense_path_array[:, 2]

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot(xe, ye, ze, label="Enemy missile trajectory", color='blue')
        ax.plot(xd, yd, zd, label="Defense missile trajectory", color='red')
        ax.scatter(xe[0], ye[0], ze[0], color='blue', s=50, label="Enemy Start")
        ax.scatter(xd[0], yd[0], zd[0], color='red', s=50, label="Defense Start")
        ax.set_xlabel('X coordinate (m)')
        ax.set_ylabel('Y coordinate (m)')
        ax.set_zlabel('Z coordinate (m)')
        ax.set_title('Missile Trajectory')
        ax.legend()
        plt.show()

# env = missile_interception_3d()
# env.reset()
# env.graph_initial()
# # env.render()

# # # # Simulation loop
# while not env.done:
#     print("-------------------------")
#     print("t: ", env.t)
#     print("enemy_x: ", env.enemy_x)
#     print("enemy_y: ", env.enemy_y)
#     print("enemy_z: ", env.enemy_z)
#     print("defense_x: ", env.defense_x)
#     print("defense_y: ", env.defense_y)
#     print("defense_z: ", env.defense_z)
#     print("-------------------------")
#     env.step([0, 0])  # Placeholder action, ProNav handles turning
#     env.t += env.time_step

# # # env.render()

num_simulations = 1000
final_relative_distances = []
success_count = 0

for _ in range(num_simulations):
    env = missile_interception_3d()
    env.reset()

    while not env.done:
        # Calculate relative distance
        relative_distance = np.linalg.norm([
            env.enemy_x - env.defense_x,
            env.enemy_y - env.defense_y,
            env.enemy_z - env.defense_z
        ])
        # Adjust time step based on relative distance
        env.adjust_time_step(relative_distance)
        env.step([0, 0])  # Placeholder action, ProNav handles turning
        env.t += env.time_step

    # print the minimum relative distance
    print("#########################################################")
    print("#########################################################")
    print("#########################################################")
    print("#########################################################")
    print("Missile Hit")
    print("Minimum relative distance: ", min(env.relative_distances))
    print("#########################################################")
    print("#########################################################")
    print("######################## #################################")
    print("#########################################################\n")


    # plt.figure()
    # plt.plot(env.times, env.relative_distances, label='Relative Distance')
    # plt.xlabel('Time (s)')
    # plt.ylabel('Distance (m)')
    # plt.title('Relative Distance Between Missiles Over Time')
    # plt.legend()
    # plt.grid(True)
    # plt.show()

    final_relative_distances.append(env.relative_distances[-1])
    if env.success:
        success_count += 1

# Calculate average final relative distance
average_relative_distance = np.mean(final_relative_distances)
success_rate = (success_count / num_simulations) * 100

print(f"Average final relative distance over {num_simulations} simulations: {average_relative_distance:.2f} meters")
print(f"Interception success rate: {success_rate:.2f}%")